In [ ]:
#! pip install matplotlib==3.4.3
#! pip install networkx==2.6.3
#! pip install pandas==1.4.1
#! pip install pyspark==3.2.0
#! pip install graphframes==0.6
#! pip install leidenalg==0.9.1

  Using cached matplotlib-3.4.3-cp38-cp38-win_amd64.whl.metadata (5.7 kB)
  Using cached pyparsing-3.1.4-py3-none-any.whl.metadata (5.1 kB)
Using cached matplotlib-3.4.3-cp38-cp38-win_amd64.whl (7.1 MB)
   ---------------------------------------- 0.0/14.9 MB ? eta -:--:--
    --------------------------------------- 0.3/14.9 MB ? eta -:--:--
   ------ --------------------------------- 2.4/14.9 MB 8.4 MB/s eta 0:00:02
   ----------- ---------------------------- 4.5/14.9 MB 9.3 MB/s eta 0:00:02
   ------------------- -------------------- 7.1/14.9 MB 9.9 MB/s eta 0:00:01
   --------------------------- ------------ 10.2/14.9 MB 11.2 MB/s eta 0:00:01
   ---------------------------------- ----- 12.8/14.9 MB 11.7 MB/s eta 0:00:01
   ---------------------------------------- 14.9/14.9 MB 11.7 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.6 MB ? eta -:--:--
   ---------------------------------------- 2.6/2.6 MB 12.2 MB/s eta 0:00:00
Using cached pyparsing-3.1.4-py3-none-any.w

In [9]:
import os
import shutil
from io import StringIO
from datetime import datetime, timedelta

import igraph as ig
import networkx as nx
import pandas as pd
import leidenalg as la
import polars as pl

#from pyspark.sql import functions as sf
#from pyspark import SparkConf
#from pyspark.sql import SparkSession
#from graphframes import GraphFrame

In [ ]:
cwd = os.getcwd()
df = pl.read_parquet(cwd + '/../HI-Small_Trans_Subset.parquet')

#subset = df.head(300000)

# Specify the output path for the new Parquet file
#output_path = cwd + '/../HI-Small_Trans_Subset.parquet'

# Write the subset DataFrame to a Parquet file
#subset.write_parquet(output_path)

#print(f"Subset saved to {output_path}")

Subset saved to c:\Users\blanc\OneDrive\Desktop\DSDM\2-Networks\networks_final\networks_final_project/../HI-Small_Trans_Subset.parquet


In [ ]:
df = df.with_row_count(name="id")
df.head()

C:\Users\blanc\AppData\Local\Temp\ipykernel_24960\3410914437.py:1: DeprecationWarning: `DataFrame.with_row_count` is deprecated. Use `with_row_index` instead. Note that the default column name has changed from 'row_nr' to 'index'.
  df = subset.with_row_count(name="id")


id,Timestamp,From Bank,Account,To Bank,Account.1,Amount Received,Receiving Currency,Amount Paid,Payment Currency,Payment Format,Is Laundering
u32,str,i64,str,i64,str,f64,str,f64,str,str,i64
0,"""2022/09/01 00:20""",10,"""8000EBD30""",10,"""8000EBD30""",3697.34,"""US Dollar""",3697.34,"""US Dollar""","""Reinvestment""",0
1,"""2022/09/01 00:20""",3208,"""8000F4580""",1,"""8000F5340""",0.01,"""US Dollar""",0.01,"""US Dollar""","""Cheque""",0
2,"""2022/09/01 00:00""",3209,"""8000F4670""",3209,"""8000F4670""",14675.57,"""US Dollar""",14675.57,"""US Dollar""","""Reinvestment""",0
3,"""2022/09/01 00:02""",12,"""8000F5030""",12,"""8000F5030""",2806.97,"""US Dollar""",2806.97,"""US Dollar""","""Reinvestment""",0
4,"""2022/09/01 00:06""",10,"""8000F5200""",10,"""8000F5200""",36682.97,"""US Dollar""",36682.97,"""US Dollar""","""Reinvestment""",0


In [32]:
names = {'id': 'id',
         'Timestamp': 'date',
         'From Bank': 'from_bank',
         'Account': 'from_account',
         'To Bank': 'to_bank',
         'Account.1': 'to_account',
         'Amount Received': 'amount_received',
         'Receiving Currency': 'currency_received',
         'Amount Paid': 'amount_sent',
         'Payment Currency': 'currency_sent',
         'Payment Format': 'payment_method',
         'Is Laundering': 'is_laundering'}

def rename_columns(df, names):
    return df.rename(names)

def timestamp_date(df):
    """Transforms date into datetime format + makes a timestamp column (Unix timestamp)"""
    df = df.with_columns(
        pl.col("date")
        .str.strptime(pl.Datetime, "%Y/%m/%d %H:%M")  # Convert string to Datetime
        .dt.timestamp()  # Convert Datetime to Unix timestamp
        .alias("timestamp"))
  
    df = df.with_columns(
        pl.col("date")
        .str.strptime(pl.Datetime, "%Y/%m/%d %H:%M")
        .dt.date())
    return df

In [ ]:
df = rename_columns(df, names)
df = timestamp_date(df)

In [41]:
df = df.with_columns((pl.col("from_account") == pl.col("to_account")).alias('same_account'))
df.group_by("payment_method").agg(
    ((pl.col("same_account").sum() / pl.count()) * 100)
    .alias("percentage_same_account"))

C:\Users\blanc\AppData\Local\Temp\ipykernel_24960\1374560205.py:3: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
  ((pl.col("same_account").sum() / pl.count()) * 100)


payment_method,percentage_same_account
str,f64
"""ACH""",8.292335
"""Credit Card""",0.162296
"""Reinvestment""",100.0
"""Bitcoin""",0.0
"""Cash""",0.450113
"""Wire""",0.340426
"""Cheque""",0.247103
